In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
# initialise data of lists.
data = {'Name':['Pieter Bartlett', 'George Berg', 'Hiroshi Motoda', 'Filippo Neri', 'Robert Roos', 'Satinder Singh', 'Maja Mataric', 'Arun Sharma', 'Michael Meystel'],
 'lenFirst':['Yes','Yes','Yes','Yes','Yes','Yes','No','No','Yes'],
 'lenLast':['Yes','No','Yes','No','No','No','Yes','Yes','Yes'],
 'sameFirst':['No','No','No','No','Yes','Yes','Yes','No','Yes'],
 'vowel':['e','e','o','i','o','i','a','a','e'],
 'Badge':['+','+','+','+','+','+','-','-','+']}
 
# Create DataFrame
df = pd.DataFrame(data)
df = df.set_index('Name')

In [4]:
def calculate_entropy(predicted_column):
    total = len(predicted_column)
    counts = predicted_column.value_counts().to_list()
    if len(counts) > 0:
        entropy = -sum((count/total) * math.log2(count/total) for count in counts)
    return entropy


In [5]:
def calculate_info_gain(df, info_gain_column_name, predicted_column_name):
    total_entropy = calculate_entropy(df[predicted_column_name])
    weighted_entropies = []
    for value in df[info_gain_column_name].unique():
        subset_entropy = calculate_entropy(df[df[info_gain_column_name] == value][predicted_column_name])
        weight = len(df[df[info_gain_column_name] == value])/len(df)
        weighted_entropies.append(subset_entropy * weight)
    info_gain = total_entropy - sum(weighted_entropies)
    return info_gain

In [ ]:
def ID3(df, original_df, features, predicted_column, parent_node):
    if len(df[predicted_column].unique()) <= 1:
        return df[predicted_column].unique()[0]

    elif len(df) == 0:
        return max(original_df[predicted_column].value_counts())
    
    else:
        pass
        #return max(df[predicted_column].value_counts())
    

    gains = [calculate_info_gain(df, feature, predicted_column, parent_node) for feature in features]
    best_feature = max(gains)

    tree = {best_feature: {}}

    #remove best feature from feature list

    for value in df[best_feature].unique():
         #value = value
         sub_df = df[df[best_feature] == value]
         sub_tree = ID3(sub_df, df, features, predicted_column, parent_node)

In [6]:
output = calculate_entropy(df['Badge'])
display(output)

stuff = calculate_info_gain(df, 'lenFirst', 'Badge')


print(stuff)



0.7642045065086203

0.7642045065086203
